In [ ]:
from pathlib import Path
import json
import re

exploration_paths = list(Path("<YOUR EXPLORATION PATH>").glob("*.json"))

interaction_log = {} # environment -> list of interactions
for exploration_path in exploration_paths:
    environment = re.search(r"multi_turn_(.*)_result.json", exploration_path.name).group(1)
    with open(exploration_path, "r") as f:
        exploration_data = [json.loads(line) for line in f]
    interaction_log[environment] = exploration_data

for environment in interaction_log.keys():
    print(f"{environment}: {len(interaction_log[environment])}")

In [ ]:
import re
from openai import OpenAI
import json

def format_interaction_log(interaction_log, environment, task_idx):
    goal = interaction_log[environment][task_idx]['inference_log'][1]['begin_of_turn_query'][1]['content'].replace("Goal: ", "").strip()
    output_str = "Environment:\n" + environment + "\nGoal:\n" + goal + "\nInteraction Log:\n"
    for step_key, step_data in interaction_log[environment][task_idx]['inference_log'][1].items():
        if step_key == 'begin_of_turn_query':
            continue
        if len(step_data) > 0:
            step = int(re.search(r"step_(\d+)", step_key).group(1)) + 1
            output_str += f"Step {step}:\n"
            if len(step_data) == 2:
                # errorneous step
                continue
            elif len(step_data) == 3:
                output_str += f"Assistant: {step_data[0]['content']}\n"
                output_str += f"Environment: {step_data[2]['content']}\n"
            else:
                raise ValueError(f"Unknown step data length: {len(step_data)}")
    return output_str


client = OpenAI(api_key=json.load(open("./secrets.json"))['openai'])
system_prompt = open("./bfcl_prompt/env_dynamics.txt").read()
def get_llm_message(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [ ]:
from tqdm import tqdm
import re

environment_dynamics = {} # environment -> list of dynamics
for environment in tqdm(interaction_log.keys()):
    environment_dynamics[environment] = []
    for task_idx in tqdm(range(len(interaction_log[environment]))):
        prompt = format_interaction_log(interaction_log, environment, task_idx)
        temp_dynamics = get_llm_message(prompt)
        try:
            # Handle possible ```json ... ``` code block or extra text
            # Try to extract the first JSON array in the string
            match = re.search(r"\[.*\]", temp_dynamics, re.DOTALL)
            if match:
                temp_dynamics = match.group(0)
            temp_dynamics = json.loads(temp_dynamics)
        except:
            print(f"Error parsing dynamics for environment {environment} and task {task_idx}")
        environment_dynamics[environment].append(temp_dynamics)

### Post processing

In [ ]:
class_to_environment = {
    'TravelAPI': 'travel_booking',
    'GorillaFileSystem': 'gorilla_file_system',
    'TradingBot': 'trading_bot',
    'MessageAPI': 'message_api',
    'TwitterAPI': 'posting_api',
    'MathAPI': 'math_api',
    'TicketAPI': 'ticket_api',
    'VehicleControlAPI': 'vehicle_control'
}
reverse_class_to_environment = {v: k for k, v in class_to_environment.items()}

# Change the key name using reverse_class_to_environment
new_environment_dynamics = {}
for env in environment_dynamics.keys():
    new_key = reverse_class_to_environment[env]
    # Flatten the list of lists into a single list for each environment
    new_environment_dynamics[new_key] = [item for sublist in environment_dynamics[env] for item in (sublist if isinstance(sublist, list) else [sublist])]

In [ ]:
cleaned_environment_dynamics = {}
for env, dynamics in new_environment_dynamics.items():
    cleaned_environment_dynamics[env] = []
    for dynamic in dynamics:
        if isinstance(dynamic, dict):
            cleaned_environment_dynamics[env].append(dynamic)
    print(f"{env}: {len(cleaned_environment_dynamics[env])}")

### Clean dynamics

In [ ]:
from openai import OpenAI
import json

client = OpenAI(api_key=json.load(open("./secrets.json"))['openai'])
system_prompt = open("./bfcl_prompt/filter_env_dynamics.txt").read()
def get_llm_message(prompt):
    response = client.chat.completions.create(
        model="o3",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
    )
    input_cost = response.usage.prompt_tokens * 2.0 / 1_000_000
    output_cost = response.usage.completion_tokens * 8.0 / 1_000_000
    cost = input_cost + output_cost
    print(f"Total cost: ${cost:.4f}")
    return response.choices[0].message.content

In [ ]:
from tqdm import tqdm

with open("environment_dynamics.json", "r") as f:
    environment_dynamics = json.load(f)

cleaned_environment_dynamics = {}
for environment, dynamics in tqdm(environment_dynamics.items()):
    print(f"{environment}: there are {len(dynamics)} dynamics")
    dynamics_str = json.dumps(dynamics)
    cleaned_dynamics = get_llm_message(f"Environment: {environment}\nDynamics: {dynamics_str}")
    try:
        if "```json" in cleaned_dynamics:
            cleaned_dynamics = json.loads(cleaned_dynamics.replace("```json\n", "").replace("```", ""))
        else:
            cleaned_dynamics = json.loads(cleaned_dynamics)
    except:
        print(f"Error parsing dynamics for environment {environment}")
    cleaned_environment_dynamics[environment] = cleaned_dynamics